# REF Assessment Notebook

### 31st May 2021

### Neil D. Lawrence

This notebook is a helper for reviewing ref papers.


In [1]:
import ref

In [2]:
df = ref.assess.data()


In [3]:
write_df = ref.access.outputs()

In [4]:
index = 251931

In [5]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [6]:
import pandas as pd

In [13]:
comment = {}
score = {}

def update_index(df, index):
    if index not in df.index:
        raise ValueError("Invalid index")
    ref.assess.query(df, index)

def update_df(df, write_df,
              o_text, o_score,
              s_text, s_score,
              r_text, r_score, interdisciplinary_comment, my_comment):
    comment = "{o_text} {o_score} {s_text} {s_score} {r_text} {r_score}".format(o_text=o_text, o_score=o_score, 
                                                                                s_text=s_text, s_score=s_score, 
                                                                                r_text=r_text, r_score=r_score)
    score = o_score + s_score + r_score
    print(comment)
    write_df.at[index, 'Comment'] = comment
    write_df.at[index, 'Score'] = score
    if interdisciplinary_comment is not None:
        write_df.at[index, 'Comment 2'] = interdisciplinary_comment
    #if my_comment is not None:
    #    df.at[index, 'Comment 4'] = str(my_comment)
    print(score)
    print(comment)

index_select = widgets.Dropdown(options=df.index[df['Interdisciplinary']=="Yes"], value=index)

o_text = "O: "
s_text = "S: "
r_text = "R: "
o_score = 2
s_score = 2
r_score = 2

import re
group = re.match(r'^O:\s*(.*)\s*([0-4])\s*S:\s*(.*)\s*([0-4])\s*R:\s*(.*)\s*([0-9])$', 
                 df['Comment'][index])
if group is not None:
    o_text = "O: " + group.group(1)
    s_text = "S: " + group.group(3)
    r_text = "R: " + group.group(5)
    o_score = int(group.group(2))
    s_score = int(group.group(4))
    r_score = int(group.group(6))
    

o_score_range = widgets.IntSlider(min=0, max=4, step=1, value=o_score)    
s_score_range = widgets.IntSlider(min=0, max=4, step=1, value=s_score)    
r_score_range = widgets.IntSlider(min=0, max=4, step=1, value=r_score)    

    

interdisciplinary_comment = df['Comment 2'][index]

interact_manual(update_index, index=index_select, df=fixed(df), write_df=fixed(write_df))


_ = interact_manual(update_df, o_text=o_text, o_score=o_score_range, 
         s_text=s_text, s_score=s_score_range, 
         r_text=r_text, r_score=r_score_range,
         interdisciplinary_comment="",
         my_comment="",
        df=fixed(df), index=fixed(index), write_df=fixed(df))







interactive(children=(Dropdown(description='index', index=57, options=(180784, 181249, 181562, 181704, 183845,…

interactive(children=(Text(value='O: ', description='o_text'), IntSlider(value=2, description='o_score', max=4…

In [16]:
df = ref.access.outputs()

In [18]:
df.loc[180784]['Comment']

KeyError: 180784

In [9]:
write_df['Comment']

0      O: integrative modelling with ontologies and a...
1      O: Sensible combination of deep learning metho...
2                                                    NaN
3                                                    NaN
4                                                    NaN
                             ...                        
837                                                  NaN
838                                                  NaN
839                                                  NaN
840                                                  NaN
841                                                  NaN
Name: Comment, Length: 842, dtype: object

In [9]:
%debug

> /Users/neil/opt/anaconda3/lib/python3.8/re.py(191)match()
    189     """Try to apply the pattern at the start of the string, returning
    190     a Match object, or None if no match was found."""
--> 191     return _compile(pattern, flags).match(string)
    192 
    193 def fullmatch(pattern, string, flags=0):

ipdb> pattern
'^O:\\s*(.*)\\s*([0-4])\\s*S:\\s*(.*)\\s*([0-4])\\s*R:\\s*(.*)\\s*([0-9])$'
ipdb> flags
0
ipdb> pattern is str
False
ipdb> q


In [ ]:
group.group(5)

In [ ]:
widgets.Dropdown?

In [ ]:
df.loc[index]['Comment']

In [ ]:
ref.assess.view(df)

In [ ]:
df['Comment 2']

In [ ]:
ref.assess.query(df, index)